In [1]:
from graph_var import PangenomeGraph
import numpy as np
import networkx as nx
from graph_var import edge_complement, node_complement, group_walks_by_name


In [8]:
gfa_file = "/Users/loconnor/Downloads/smaller_sub.gfa"
# gfa_file = "/Users/loconnor/Downloads/grch38_chr20.gfa"
# gfa_file = "../data/c4a_with_inversion.gfa"
# gfa_file = "../data/c4a.gfa"
G, walks, walk_sample_names = PangenomeGraph.from_gfa(gfa_file, reference_path_index=None, return_walks=True)

Reference walk: 1
Num of binodes: 185471
Num of biedges: 259618
Assigning node directions
Computing reference tree
Computing branch points
Computing positions


In [9]:
haplotypes = group_walks_by_name(walks, walk_sample_names)
# haplotypes.keys()

In [10]:
%%time
for sample_name in np.unique(walk_sample_names):
    haplo = haplotypes[sample_name]
    missing, missing2 = G.get_missing_variants(haplo)
    assert len(missing) == len(missing2)
    if missing:
        assert missing[0] in missing2
        assert missing2[0] in missing

CPU times: user 19.9 s, sys: 217 ms, total: 20.1 s
Wall time: 20 s


In [12]:
for x in missing:
    assert x in missing2

In [5]:
result = []
for sample_name in np.unique(walk_sample_names):
    haplo = haplotypes[sample_name]
    missing = G.get_missing_variants(haplo)
    for geno in [G.genotype(x) for x in haplo]:
        for key in list(geno.keys()):
            if key in missing: 
                result.append(key)
assert not result

In [12]:
%%time
result = G.sorted_variant_edge()

CPU times: user 115 ms, sys: 9.44 ms, total: 125 ms
Wall time: 124 ms


In [10]:
G.position('40967991_+'), G.right_position('40967992_+')

(6038834, 6038952)

In [30]:
from graph_var.utils import walk_complement
def get_missing_variants(self, walks: list, variant):

    # walks must be in the positive direction
    for i, walk in enumerate(walks):
        if self.nodes[walk[0]]['direction'] == -1:
            walk = walk_complement(walk)
            walks[i] = walk
        assert self.nodes[walk[-1]]['direction'] == 1, "encountered a walk with an odd number of inversion"
        assert self.position(walk[0]) <= self.right_position(walk[-1])
    
    for walk in walks:
        print(walk[0], walk[-1])

    # order walks by position
    source_positions = np.sort([self.nodes[walk[-1]]['right_position'] for walk in walks] + [self.position('+_terminus_+')])
    sink_positions = np.sort([self.nodes[walk[0]]['position'] for walk in walks] + [self.position('-_terminus_+')])
    print(source_positions, sink_positions)
    u, v = variant
    left_u = self.position(u)
    right_v = self.right_position(v)
    print(left_u, right_v)

    # such that source_positions[index-1] <= left_u
    index = np.searchsorted(source_positions, left_u, side='right')
    print(index)
    
    return right_v <= sink_positions[index-1]


In [31]:
get_missing_variants(G, haplo, ('40967991_+', '40967992_+'))

40781781_+ 40967993_+
[      0 6038953] [  60290 6038953]
6038834 6038952
1


False

In [27]:
G.position('40781781_+')

60290

In [12]:
G.right_position(haplo[0][0]), G.position(haplo[0][-1])

(6038953, 60290)

In [9]:
G.nodes[haplo[0][0]]['direction']

-1

In [4]:
c = nx.find_cycle(positive_subgraph)
G.edges[c[0]], c

({'index': 4.0,
  'weight': 10,
  'is_in_tree': False,
  'branch_point': 'STK19_+',
  'is_representative': True},
 [('C4A_+', 'C4A_+')])

In [11]:
from graph_var.utils import group_walks_by_name
walks_by_name = group_walks_by_name(walks, walk_sample_names)

NameError: name 'defaultdict' is not defined

In [8]:
self = G

In [10]:
walk_position_intervals = [(self.nodes[walk[0]]['position'], self.nodes[walk[-1]]['position'])
        for walk in walks]
walk_position_intervals

[(60290, 6038953),
 (50126, 6038953),
 (819242, 6038953),
 (60290, 817033),
 (822406, 6038953),
 (60290, 817772),
 (60290, 6038953),
 (60290, 6038953),
 (6038953, 60290),
 (6038953, 819240),
 (817523, 60290),
 (6038953, 820113),
 (818810, 60290),
 (60290, 6038953),
 (60290, 6038953),
 (6038953, 60290),
 (819825, 3825592),
 (4104067, 6038953),
 (818810, 60290),
 (4100072, 3827157),
 (4102199, 6038953),
 (4099933, 60290),
 (6038953, 2822514),
 (60290, 818564),
 (2815941, 823491),
 (6038953, 60290),
 (60290, 6038953),
 (6038953, 60290),
 (822573, 6038953),
 (60290, 818810),
 (389608, 6038953),
 (403930, 60290),
 (60290, 6038953),
 (6038953, 60290),
 (6038953, 2827999),
 (818550, 60290),
 (2827462, 819325),
 (819240, 6038953),
 (818810, 60290),
 (6038953, 60290),
 (4032678, 6038953),
 (60290, 4031884),
 (6038953, 60290),
 (820246, 6038953),
 (818810, 60290),
 (6038953, 820049),
 (818810, 60290),
 (820049, 6038953),
 (818379, 60290),
 (60290, 6038953),
 (6038953, 60290),
 (6038953, 60290),


In [5]:
G.variant_edges_summary()

100%|██████████| 6/6 [00:00<00:00, 44699.51it/s]


{'inversion': 0,
 'crossing_edges': 0,
 'back_edges': 4,
 'forward_edges': 2,
 'total': 6}

In [11]:
G.variant_edges_summary()

100%|██████████| 8/8 [00:00<00:00, 29615.56it/s]


{'inversion': 2,
 'crossing_edges': 4,
 'back_edges': 0,
 'forward_edges': 2,
 'total': 8}

In [ ]:
node = walks[5][100]
G.nodes[node]['distance_from_reference']

In [ ]:
visits = [G.count_edge_visits(genotype) for genotype in genotypes]

In [ ]:
allele_counts = G.allele_count()

In [ ]:
allele_counts

In [ ]:
vcf_filename = "chr20_subgraph.vcf"
tree_filename = "chr20_subgraph.tree"
chr_name = "chr20"
G.write_vcf(walks, walk_sample_names, vcf_filename, tree_filename, chr_name)

In [ ]:
cpts = nx.weakly_connected_components(G)
sub = nx.subgraph(G, next(cpts))
pos = nx.spring_layout(sub, k=10)
nx.draw(sub, pos, with_labels=True)

In [ ]:
pos = nx.spring_layout(G.reference_tree, k=10)
nx.draw(G.reference_tree, pos, with_labels=True)

In [ ]:
for u in G.nodes:
    G.nodes[u]['sequence'] = u


for e in G.variant_edges:
    if G.is_inversion(e):
        continue
    ref, alt, _, branch = G.ref_alt_alleles(e)
    print(f"edge: {e}, ref: {ref}, alt: {alt}, branch: {branch}")
    ref, alt, _, branch = G.ref_alt_alleles(_edge_complement(e))
    print(f"edge: {_edge_complement(e)}, ref: {ref}, alt: {alt}, branch: {branch}")

In [ ]:
invs = [e for e in G.variant_edges if G.is_inversion(e)]
for e in invs:
    ref, alt, _, branch = G.ref_alt_alleles(e)
    print(f"edge: {e}, ref: {ref}, alt: {alt}, branch: {branch}")
    ref, alt, _, branch = G.ref_alt_alleles(_edge_complement(e))
    print(f"edge: {_edge_complement(e)}, ref: {ref}, alt: {alt}, branch: {branch}")

In [ ]:
ref, alt, _, branch = G.ref_alt_alleles(invs[0])
ref,alt,branch

In [ ]:
[len(w) for w in walks]
walks[6]

In [ ]:
G.walk_with_variants('STK19_+','C4B_-',[])

In [ ]:
first, last, variant_edges = 'C4A_-','STK19_+',[('C4A_-', 'CYP21A2_+')]
walk_direction = G.direction(first)
source_nodes = [first] + [v for _, v in variant_edges]
sink_nodes = [u for u, _ in variant_edges] + [last]
source_nodes, sink_nodes

In [ ]:
walk_direction, G.direction(source_nodes[1])

In [ ]:
segment, _ = G.walk_up_tree('STK19_+','CYP21A2_+')
segment[1:]

In [ ]:
segment[::-1]

In [ ]:
G.walk_with_variants('C4A_-','STK19_+',[invs[1]])


In [ ]:
ref, alt, _, branch = G.ref_alt_alleles(_edge_complement(invs[1]))
ref,alt,branch

In [ ]:
invs[1]

In [ ]:
# G.ref_alt_alleles(_edge_complement(e))
u,v = _edge_complement(e)
branch_point = G.edges[u, v]['branch_point']
first, last = (branch_point, v) if G.nodes[u]['direction'] == 1 else (v, branch_point)
first,last
# G.nodes[e[0]], G.nodes[e[1]], _edge_complement(e), G.edges[_edge_complement(e)]['branch_point']

In [ ]:
u,v

In [ ]:
_edge_complement(e)

In [ ]:
G.walk_with_variants('CYP21A2_-', 'C4A_+', [_edge_complement(e)])

In [ ]:
e = ('C4B_+', 'C4A_+')
u, v = e
branch_point = G.edges[u, v]['branch_point']
ref, alt, _, branch = G.ref_alt_alleles(e)
print(f"edge: {e}, ref: {ref}, alt: {alt}, branch: {branch}")

In [ ]:
alt_path = G.walk_with_variants(branch_point, v, [e])
alt_path

In [ ]:
l = [1]
l += [2]
l[1:-1]

In [ ]:
sink_nodes = [u] + [v]
source_nodes = [branch_point] + [v]
source_nodes, sink_nodes

In [ ]:
G.nodes['C4A_+']['direction']

In [ ]:
segment, _ = G.walk_up_tree('C4A_+', 'C4B_+')
segment[::-1]

In [ ]:
ref_path = G.walk_with_variants('C4A_+', 'C4A_+', [])
G.walk_sequence(ref_path)

In [ ]:
G.ref_alt_alleles(('C4A_+', 'C4A_+'))

In [ ]:
G.nodes['C4A_+']

In [ ]:
for e in G.variant_edges:
    ref, alt, _, _ = G.ref_alt_alleles(_edge_complement(e))
    print(f"ref: {ref}, alt: {alt}")

In [ ]:
first = 'C4A_-'
last = 'DXO_-'
edge = (first, last)
ref = G.walk_with_variants(*edge, [])
alt = G.walk_with_variants(*edge, [edge])
ref, alt

In [ ]:
first = 'SKIC2_+'
last = 'STK19_-'
edge = (first, last)
ref = G.walk_with_variants(first, last, [])
alt = G.walk_with_variants(first, last, [edge])
ref, alt

In [ ]:
first = 'C4A_-'
last = 'DXO_-'
edge = (first, last)
# edge = _edge_complement(edge)
ref = G.walk_with_variants(*edge, [])
alt = G.walk_with_variants(*edge, [edge])
ref, alt

In [ ]:
G.nodes[first]['direction'], G.nodes[last]['direction']

In [ ]:
G.edges[first, last]['branch_point']

In [ ]:
first = 'C4A_+'
last = 'CYP21A2_+'
edge = _edge_complement((first, last))
ref = G.walk_with_variants(*edge, [])
alt = G.walk_with_variants(*edge, [edge])
ref, alt

In [ ]:
first = 'C4A_+'
last = 'CYP21A2_+'
edge = (first, last)
ref = G.walk_with_variants(*edge, [])
alt = G.walk_with_variants(*edge, [edge])
ref, alt

In [ ]:
G.walk_up_tree(last, last)

In [ ]:
walks[0]

In [ ]:
classification = []

endpoints = "STK19", "CYP21A2"
variants = [("STK19_+", "C4B_+"), ("C4A_+", "CYP21A2_+")]
classification.append(G.classify_triallelic_bubble(endpoints, variants))

endpoints = "C4A", "CYP21A2"
variants = [("C4B_+", "C4B_+"), ("C4A_+", "CYP21A2_+")]
classification.append(G.classify_triallelic_bubble(endpoints, variants))

endpoints = "C4A", "CYP21A2"
variants = [("CYP21A2_+", "C4B_+"), ("C4A_+", "CYP21A2_+")]
classification.append(G.classify_triallelic_bubble(endpoints, variants))

classification

In [ ]:
inversions = [e for e in G.variant_edges if G.is_inversion(e)]
inversions

In [ ]:
from utils import _node_complement, _edge_complement
u,v = inversions[0]
G.edges[u,_node_complement(v)]['branch_point']

In [ ]:
u,v

In [ ]:
def annotate_branch_points(self) -> None:
    """
    Computes the branch point, i.e. the lowest common ancestor in the reference tree, of each variant edge.
    """

    def positive_node(node):
        return node if self.nodes[node]['direction'] == 1 else _node_complement(node)

    positive_direction_variants = [(positive_node(u), positive_node(v)) for u,v in self.variant_edges]

    # For each variant edge (u,v), yield the lowest common ancestor of u and v in the tree
    branch_point_tuples = nx.tree_all_pairs_lowest_common_ancestor(
        self.reference_tree,
        root=self.termini[0]+'_+',
        pairs=positive_direction_variants
    )
    branch_points = dict(branch_point_tuples)

    for edge in self.variant_edges:
        branch_point = branch_points[positive_node(edge[0]), positive_node(edge[1])]
        self.edges[edge]['branch_point'] = branch_point
        self.edges[_edge_complement(edge)]['branch_point'] = branch_point

In [ ]:
annotate_branch_points(G)

In [ ]:
u,v = inversions[0]
G.edges[u,v]['branch_point'], u, v

In [ ]:
u,v = inversions[1]
G.edges[u,v]['branch_point'], u, v

In [ ]:
inversions

In [ ]:
G.nodes['42737147_+']['direction']

In [ ]:
G.has_edge("CYP21A2_+", "C4B_+")

In [ ]:
G.reference_tree.edges

In [ ]:
('42693511_-','42693513_+') in G.variant_edges

In [ ]:
G.is_snp(('42693511_-','42693513_+'))

In [ ]:
inversions = [edge for edge in G.variant_edges if G.is_inversion(edge)]

In [ ]:
inversions

In [ ]:
walks

In [ ]:
genos = [G.genotype(walk) for walk in walks[:10]]

In [ ]:
for geno in genos:
    x = []
    for e in inversions:
        if e not in geno:
            x.append(0)
            continue
        x.append(geno[e])
    print(x)

In [ ]:
len(genos)

In [ ]:
list(G.neighbors('42693514_+'))

In [ ]:
[G.nodes[node] for node in G.neighbors('42693514_+')]

In [ ]:
G.nodes['42693514_+']

In [ ]:
e = inversions[0]
G.ref_alt_alleles(e)

In [ ]:
list(G.neighbors('42693513_-'))

In [ ]:
list(G.predecessors('42693513_+'))

In [ ]:
path = nx.shortest_path(G, '-_terminus_-','42693513_+')

In [ ]:
sum([G.nodes[node]['on_reference_path'] for node in path])

In [ ]:
G.node_attribute_names

In [ ]:
G.nodes['42693513_+']['direction']

In [ ]:
edgeinfo_file = "/Users/loconnor/Downloads/smaller_sub.edge"
nodeinfo_file = "/Users/loconnor/Downloads/smaller_sub.node"
G.write_edgeinfo(edgeinfo_file)
G.write_nodeinfo(nodeinfo_file)

In [ ]:
G2 = PangenomeGraph.from_gfa(gfa_file, reference_path_index=1, nodeinfo_file=nodeinfo_file, edgeinfo_file=edgeinfo_file)

In [ ]:
G3 = PangenomeGraph.from_gfa(gfa_file, reference_path_index=1, edgeinfo_file=edgeinfo_file)

In [ ]:
G4 = PangenomeGraph.from_gfa(gfa_file, reference_path_index=1, nodeinfo_file=nodeinfo_file)

In [ ]:
np.sum([pos for _, pos in G.nodes(data='position')])

In [ ]:
np.sum([pos for _, pos in G4.nodes(data='position')])

In [ ]:
G2.nodes['-_terminus_+']['position'], G.nodes['-_terminus_+']['position']

In [ ]:
fh = open(gfa_file)
next(fh)

In [ ]:
fh = open(gfa_file)
for line in fh:
    parts = line.strip().split('\t')
    if parts[0] == 'S':
        continue
    break
print(line)

In [ ]:
def subset_gfa_to_tree(gfa_file, output_file, G):
    with open(gfa_file) as fh:
        with open(output_file, 'w') as fw:
            for line in fh:
                parts = line.strip().split('\t')
                if parts[0] == 'S':
                    fw.write(line)
                elif parts[0] == 'L':
                    edge = (f"{parts[1]}_{parts[2]}", f"{parts[3]}_{parts[4]}")
                    if G.edges[edge]['is_in_tree']:
                        fw.write(line)
                              

In [ ]:
subset_gfa_to_tree(gfa_file, "/Users/loconnor/Downloads/smaller_sub_tree.gfa", G)

In [ ]:
l = list(G.nodes)
l[1000]